## Homework 1 submitted by Abu Huraira

## 1 Symbolic differentiation

### 1.1 Polynomial expressions
In this assignment you will need to implement the tools for symbolic differentiation of expressions.
Symbolic differentiation computes derivative of a given expression symbolically (i.e. as an expression), for example, given (1 + 2x)(x + y) the computed partial derivative with respect to variable y is 1 + 2x.
    For now, we will consider only expressions involving numeric constants, variables, binary addition, and binary multiplication. The expressions will be given as valid Racket terms, for example (note the
use of quotation):
1
'(+ 1 x) ; 1 + x
'(* 2 y) ; 2y
'(* (+ 1 (* 2 x)) (+ x y)) ; (1 + 2x)(x + y)
To work with these expressions, you need to be able to traverse its structure. You can use number? predicate to check whether an expression is a number.

### Exercise 1.1 (⋆, 5 points). 

Implement the following predicates and functions:
```racketlang
(define (variable? expr) ...) ; check whether a given expression is a variable
(define (sum? expr) ...) ; check whether a given expression is a sum
(define (summand-1 expr) ...) ; extract first summand from a sum
(define (summand-2 expr) ...) ; extract second summand from a sum
(define (product? expr) ...) ; check whether a given expression is a product
(define (multiplier-1 expr) ...) ; extract first multiplier from a product
(define (multiplier-2 expr) ...) ; extract second multipler from a product
```
Now, whenever we have an expression we can check what kind of expression we have and decompose
it into its constituent subexpressions.

### Solution: 

In [75]:
(define (variable? expr)
  (symbol? expr))

(define (sum? expr)
  (and (pair? expr)
       (eq? (car expr) '+)))

(define (summand-1 expr)
  (if (sum? expr)
      (cadr expr)
      (error "Not a sum")))

(define (summand-2 expr)
  (if (sum? expr)
      (caddr expr)
      (error "Not a sum")))

(define (product? expr)
  (and (pair? expr)
       (eq? (car expr) '*)))

(define (multiplier-1 expr)
  (if (product? expr)
      (cadr expr)
      (error "Not a product")))

; 
(define (multiplier-2 expr)
  (if (product? expr)
      (caddr expr)
      (error "Not a product")))

; Test cases
(displayln (variable? 'x))
(displayln (sum? '(+ 1 x)))
(displayln (summand-1 '(+ 1 x)))
(displayln (summand-2 '(+ 1 x)))
(displayln (product? '(* 1 x)))
(displayln (multiplier-1 '(* 1 x)))
(displayln (multiplier-2 '(* 1 x)))

#t
#t
1
x
#t
1
x


### Exercise 1.2 (⋆, 5 points).
Implement a recursive function derivative that computes a symbolic derivative of a given expression with respect to a given variable. At this point you are not expected to simplify expressions after differentiation:

```lisp
(derivative '(+ 1 x) 'x)
; '(+ 0 1)
(derivative '(* 2 y) 'y)
; '(+ (* 0 y) (* 2 1))
(derivative '(* (+ x y) (+ x (+ x x))) 'x)
; '(+ (* (+ 1 0) (+ x (+ x x))) (* (+ x y) (+ 1 (+ 1 1))))

```
### Solution: 

In [76]:
(define (derivative expr var)
  (cond
    ((variable? expr)
     (if (eq? expr var)
         1
         0))
    ((number? expr) 0)
    ((sum? expr)
     (list '+ (derivative (summand-1 expr) var)
              (derivative (summand-2 expr) var)))
    ((product? expr)
     (list '+ (list '* (derivative (multiplier-1 expr) var) (multiplier-2 expr))
              (list '* (multiplier-1 expr) (derivative (multiplier-2 expr) var))))))


; Test cases
(displayln (derivative '(+ 1 x) 'x))
(displayln (derivative '(* 2 y) 'y))
(displayln (derivative '(* (+ x y) (+ x (+ x x))) 'x))

(+ 0 1)
(+ (* 0 y) (* 2 1))
(+ (* (+ 1 0) (+ x (+ x x))) (* (+ x y) (+ 1 (+ 1 1))))


### Exercise 1.3 (⋆⋆, 10 points). 
Implement a recursive function simplify that simplifies an expression using the following rules:
1. 0 + e = e for all expressions e,
2. e + 0 = e for all expressions e,
3. 1 ∗ e = e for all expressions e,
4. e ∗ 1 = e for all expressions e,
5. 0 ∗ e = 0 for all expressions e,
6. e ∗ 0 = 0 for all expressions e,
7. sums and products of numeric constants should be computed.
Examples:
```racketlang
(simplify '(+ 0 1))
; 1
(simplify '(+ (* 0 y) (* 2 1)))
; 2
(simplify '(+ (* (+ 1 0) (+ x (+ x x))) (* (+ x y) (+ 1 (+ 1 1)))))
; '(+ (+ x (+ x x)) (* (+ x y) 3))
```
*Hint: it might be easier to implement a non-recursive function* `simplify-at-root` *that only simplifies expression if it matches exactly left-hand side of one of the rules. Then use simplify-at-root to implement a recursive function simplify.*

### Solution:

In [77]:
(define (simplify-at-root expr)
  (cond
    ((and (sum? expr)
          (number? (summand-1 expr))
          (number? (summand-2 expr)))
     (+ (summand-1 expr) (summand-2 expr)))
    ((and (product? expr)
          (number? (multiplier-1 expr))
          (number? (multiplier-2 expr)))
     (* (multiplier-1 expr) (multiplier-2 expr)))
    ((and (eq? (car expr) '+)
          (eq? (cadr expr) 0))
     (simplify-at-root (caddr expr)))
    ((and (eq? (car expr) '+)
          (eq? (caddr expr) 0))
     (simplify-at-root (cadr expr)))
    ((and (eq? (car expr) '*)
          (eq? (cadr expr) 1))
     (simplify-at-root (caddr expr)))
    ((and (eq? (car expr) '*)
          (eq? (caddr expr) 1))
     (simplify-at-root (cadr expr)))
    ((and (eq? (car expr) '+)
          (eq? (cadr expr) 0)
          (eq? (caddr expr) 0))
     0)
    ((and (eq? (car expr) '*)
          (or (eq? (cadr expr) 0)
              (eq? (caddr expr) 0)))
     0)
    (else expr)))

(define (simplify expr)
  (if (or (number? expr) (variable? expr))
      expr
      (simplify-at-root (list (car expr)
                              (simplify (cadr expr))
                              (simplify (caddr expr))))))

; Test cases
(displayln (simplify '(+ 0 1)))
(displayln (simplify '(+ (* 0 y) (* 2 1))))
(displayln (simplify '(+ (* (+ 1 0) (+ x (+ x x))) (* (+ x y) (+ 1 (+ 1 1))))))


1
2
(+ (+ x (+ x x)) (* (+ x y) 3))


### Exercise 1.5 (⋆, 5 points).
Implement a recursive function to-infix that converts an expression into an infix form:
```racketlang
(to-infix '(+ (+ x (+ x x)) (* (+ x y) 3))
; '((x + (x + x)) + ((x + y) * 3)
```

### Solution: 

In [78]:
(define (to-infix expr)
  (cond
    ((number? expr) expr)
    ((variable? expr) expr)
    ((sum? expr)
     (list (to-infix (summand-1 expr))
           '+
           (to-infix (summand-2 expr))))
    ((product? expr)
     (list (to-infix (multiplier-1 expr))
           '*
           (to-infix (multiplier-2 expr))))))

; Testing to-infix
(displayln (to-infix '(+ (+ x (+ x x)) (* (+ x y) 3))))

((x + (x + x)) + ((x + y) * 3))


## 1.2 More functions
### Exercise 1.6 (⋆, 5 points). 
Update functions derivative and simplify to support the following functions (here e, e1 and e2 denote arbitary expressions):
1. exponentiation: (e1)^(e2)
2. trigonometric functions: sin (e), cos (e), tan (e)
3. natural logarithm: log (e)

### Solution: 

### Exercise 1.7 (⋆⋆, 10 points). 
Update functions derivative and simplify to support the following polyvariadic sums and products:

```racketlang
(derivative '(+ 1 x y (* x y z)) 'x)
; '(+ 0 1 0 (+ (* 1 y z) (* x 0 z) (* x y 0)))
(simplify '(+ 0 1 0 (+ (* 1 y z) (* x 0 z) (* x y 0))))
; '(+ 1 (* y z))
```

“polyvariadic” means that a function or operator can support arbitrary number of arguments; for example, + in Racket can accept as many arguments, as user wants: (+ 1 2 3 4) computes to 1 + 2 + 3 + 4 = 10.

### Solution: 

## 1.3 Gradient

### Exercise 1.8 (⋆⋆, 10 points). 
Implement a function variables-of that returns a (sorted) list of distinct variables used in a given expression:

```racketlang
(variables-of '(+ 1 x y (* x y z)))
; '(x y z)
```

In [79]:
(define (variables-of expr)
  (define (collect-variables expr variables)
    (cond
      ((variable? expr)
       (if (not (member expr variables equal?))
           (cons expr variables)
           variables))
      
      ((pair? expr)
       (foldl collect-variables variables (cdr expr)))
      
      (else variables)))
  
  (sort (collect-variables expr '()) symbol<?))

; Testing variables-of
(displayln (variables-of '(+ 1 x y (* x y z))))
(displayln (variables-of '(+ 1 2 3)))
(displayln (variables-of '(+ 1 x y (* x y z) 1 2 3)))

(x y z)
()
(x y z)


### Exercise 1.9 (⋆⋆, 10 points). 

Implement a function gradient that returns a gradient of a multivariable expression (given explicitly the list of variables). Recall that the gradient ∇f is a vector consisting of partial derivatives:


Represent gradient in Racket as a list of expressions:

```racketlang
(gradient '(+ 1 x y (* x y z)) '(x y z))
; '((+ 1 (* y z)) (+ 1 (* x z)) (* x y))
```

### Solution: 